In [1]:
from typing import Annotated
import wikipedia
from langchain_core.tools import tool 
import pandas as pd 
import numpy as np 
from alpha_vantage.timeseries import TimeSeries
import os
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
@tool 
def wikipedia_tool(query : Annotated[str, "Execute wikipedia search to find key summary information about a given topic."]) -> str :
    """Use this to search Wikipedia for factual information."""
    try:
        results = wikipedia.search(query)

        if not results:
            return "No results found on Wikipedia for this topic."
        
        title = results[0]

        summary = wikipedia.summary(title, sentences = 5, auto_suggest = False, redirect = True)

    except Exception as e :
        return f"Failed to Execute : {e}"

    result_string = f"\n\nWIKIPEDIA SUMMARY:\n{summary}\n\n"

    return result_string

In [3]:
summary = wikipedia_tool.invoke("Generative AI")

In [4]:
print(summary)



WIKIPEDIA SUMMARY:
Generative artificial intelligence (Generative AI, GenAI, or GAI) is a subfield of artificial intelligence that uses generative models to produce text, images, videos, or other forms of data. These models learn the underlying patterns and structures of their training data and use them to produce new data based on the input, which often comes in the form of natural language prompts.
Generative AI tools have become more common since the AI boom in the 2020s. This boom was made possible by improvements in transformer-based deep neural networks, particularly large language models (LLMs). Major tools include chatbots such as ChatGPT, Copilot, Gemini, Claude, Grok, and DeepSeek; text-to-image models such as Stable Diffusion, Midjourney, and DALL-E; and text-to-video models such as Veo, LTXV and Sora.




In [5]:
@tool 
def historical_stock_price_data_tool(
    company_ticker : Annotated[str, "The ticker symbol of the company to retrieve their stock histrical stock price data."],
    num_days : Annotated[int, "The number of days of stock data required to respond to the user query."]
    ) -> pd.DataFrame | str :
    """FETCHES HISTORICAL STOCK PRICE DATA FOR A GIVEN TICKER USING ALPHA VANTAGE"""
    
    ts = TimeSeries(key = os.getenv("ALPHA_VANTANGE_API_KEY"))
    data, metadata = ts.get_daily(symbol = company_ticker, outputsize = "full")

    data_df = pd.DataFrame(data)
    data_df = data_df.transpose()
    data_df.rename(columns = {"1. open" : "Open", '2. high' : "High", '3. low' : "Low", '4. close' : "Close", '5. volume' : "Volume"}, inplace = True)
    data_df.to_csv(f"{company_ticker}.csv")

    max_number_of_days = len(data_df)

    if num_days > max_number_of_days:
        return "Insufficient Data : The date range requested exceeds the dataset's date range."
    
    else:
        print(f"Successfully extracted historical stock price data for {company_ticker}")
        return data_df
    

In [9]:
df = historical_stock_price_data_tool.invoke(
    {"company_ticker": "NVDA",
    "num_days": 100}
    )

Successfully extracted historical stock price data for NVDA
